In [2]:
#Caricare le librerie
import tweepy
import pandas as pd
import numpy as np
import re
import json

In [5]:
consumer_key= "rrR0Znz98oKkjfB8b72oLH9Qc"
consumer_secret= "m7iOPX6iW8R3IalRf7x56YVeIb18o3I1bhUftg8JzlQ6fAjLgW"
access_token= "1442250607-3cJYKYyh7Ae19nMcv5YoxOIB2XlvgVX0ROaDSJx"
access_token_secret= "ZOnaaCTgO7DsKtmPjVruOlkvzBY8p6MbkKLjiYLdamBYE"
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [42]:
#Scaricare in un dataframe i tweet in base ad un hashtag (until="2021-12-21")
#NB: hasattr(obj, name, /) Return whether the object has an attribute with the given name.
hashtag="#lockdown"
list_tweet=[]
list_originals=[]
list_retweetted=[]
n=1000
for tweet in tweepy.Cursor(api.search_tweets,q=hashtag,count=100,lang="it").items(n):

  #gestione dei retweet con try-except
  try:
    list_originals.append(tweet.retweeted_status.id) #creo lista dei retweets id 
    #lista tweets che sono stati retweettati
    list_retweetted.append([tweet.retweeted_status.created_at, tweet.retweeted_status.id, tweet.retweeted_status.text, 
                     tweet.retweeted_status.favorite_count, tweet.retweeted_status.retweet_count, 
                     tweet.retweeted_status.user.screen_name]) 
  except:
    list_originals.append(0)
  #creo lista e df dei tweet  
  list_tweet.append([tweet.created_at, tweet.id, tweet.text, 
                     tweet.favorite_count, tweet.retweet_count, 
                     tweet.user.location,tweet.user.screen_name,
                     hasattr(tweet, "retweeted_status"),
                     tweet.in_reply_to_screen_name])
tweets = pd.DataFrame(list_tweet[1:], 
                      columns=['date','id','text','like','n_rt','location','author',
                               "retweet","reply_name"])
tweets["retwitted_id"]=list_originals[1:]
#creo df dei retweet
retweetted = pd.DataFrame(list_retweetted[1:], 
                      columns=['date','id','text','like','n_rt','author'])                                                    
tweets.head()

,date,id,text,like,n_rt,location,author,retweet,reply_name,retwitted_id
0,2021-12-20 22:52:45+00:00,1473063855328739334,RT @EsercitoCrucian: Noi invece Proponiamo il ...,0,365,Genova,Thelma02559007,True,None,1472956905664520192
1,2021-12-20 22:50:38+00:00,1473063321649790986,Potete avere e conoscere tutte le ragioni per ...,0,0,,Barbara79974676,False,None,0
2,2021-12-20 22:50:26+00:00,1473063270911254528,Attendibili come l'oroscopo e parlano di #lock...,0,0,"Terra di Mezzo, Beleriand",Soffrodinsomma,False,None,0
3,2021-12-20 22:49:11+00:00,1473062954669064198,Questo è nazismo criminale! Atti insopportabil...,0,0,,Barbara79974676,False,None,0
4,2021-12-20 22:49:00+00:00,1473062911513931781,RT @valy_s: Il turismo ha perso l’80% per #loc...,0,9,,Alexct9,True,None,1473060043817984006


In [24]:
#selezione dei tweet più retwettati 
retweetted.sort_values("n_rt",ascending=False).drop_duplicates()

,date,id,text,like,n_rt,author
522,2021-12-18 19:09:39+00:00,1472282934073872397,Che fate a Capodown?\n#lockdown,4677,664,lefrasidiosho
27,2021-12-18 22:56:11+00:00,1472339940327444482,Con il 90% di #vaccinati si moltiplicano i pos...,2178,584,intuslegens
215,2021-12-18 20:23:00+00:00,1472301392467476489,Saranno intitolate strade e piazze in suo onor...,1377,475,strange_days_82
256,2021-12-20 15:47:47+00:00,1472956905664520192,Noi invece Proponiamo il #Lockdown TOTALE e de...,481,354,EsercitoCrucian
441,2021-12-18 19:36:15+00:00,1472289627205775372,Il #lockdown è ormai da considerarsi come una ...,1507,325,nonexpedit
...,...,...,...,...,...,...
60,2021-12-20 21:45:32+00:00,1473046936030699527,"I Diritti Umani sono Inalienabili, fatevene un...",1,1,Leonedellaluce
335,2021-12-20 19:26:23+00:00,1473011918000558085,Se #SalviniPagliaccio dice che farà di tutto p...,2,1,DRusconi
68,2021-12-20 21:40:41+00:00,1473045718508220419,@MMaXXprIIme @drsmoda Vaccinatevi sorci… \nanz...,3,1,tradori
55,2021-12-20 21:50:03+00:00,1473048076290072581,Questo è uno dei vergognosi sistemi di comunic...,2,1,Leonedellaluce


##### Vado a vedere quali sono i tweet più retwittati e quante volte sono stati ritwittati dalla nostra rete (usando SQL)

In [45]:
#Open the terminal and run-   pip install U pandasql
#Open your Jupyter Notebook and in any cell run-    !pip install -U pandasql
import pandasql as ps
q1 = """SELECT id,like,n_rt, author,count() FROM retweetted group by id order by n_rt desc"""
print(ps.sqldf(q1, locals()))

                     id  like  n_rt           author  count()
0   1472282934073872397  4677   664    lefrasidiosho        1
1   1472339940327444482  2178   584      intuslegens        2
2   1472301392467476489  1377   475  strange_days_82        1
3   1472956905664520192   491   365  EsercitoCrucian      165
4   1472289627205775372  1507   325       nonexpedit        3
..                  ...   ...   ...              ...      ...
77  1472955028306546688     4     1  Stefano_villa73        1
78  1472948554629062658     8     1  cesololinter_19        1
79  1472903308989849600     3     1        robkapfas        1
80  1472865299397550086     1     1         il_Mitte        1
81  1470810332654362634     0     1   BorasoGianluca        1

[82 rows x 5 columns]
